In [1]:
import pandas as pd
import torch
import os
import itertools
import uuid

import sys
sys.path.append('../')


from dataloader.dataloader import *
from training.training import *
from models.rae import *
from utils.utils import *
from visualizations.visualizations import *
from evaluation.evaluation import *

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Training on GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Training on CPU.")

CUDA is available. Training on GPU.


In [3]:
data_folder = "../Data/Data v5"


# df = pd.read_csv(os.path.join(data_folder, "amari_ue_data_final_v5_no_outliers.csv"))
# df = df.sort_values(["imeisv", "_time"], ascending = True)
# dataset_used = 'no_outliers'

df = pd.read_csv(os.path.join(data_folder, "amari_ue_data_final_v5_no_outliers_scaled.csv"))
df = df.sort_values(["imeisv", "_time"], ascending = True)
dataset_used = 'no_outliers_scaled'

# df = pd.read_csv(os.path.join(data_folder, "amari_ue_data_final_v5_no_outliers_scaled_sep.csv"))
# df = df.sort_values(["imeisv", "_time"], ascending = True)
# dataset_used = 'no_outliers_scaled_sep'

In [4]:
# feature_columns = [
#     'dl_bitrate','ul_bitrate', 
#     'cell_x_dl_retx', 'cell_x_dl_tx',
#     'cell_x_ul_retx', 'cell_x_ul_tx',
#     'ul_total_bytes_non_incr', 'dl_total_bytes_non_incr'
# ]

# feature_columns = [
#     'dl_bitrate','ul_bitrate','ul_total_bytes_non_incr', 'dl_total_bytes_non_incr'
# ]

feature_columns = [
    'ul_bitrate'
]

In [5]:
rolling_avg_label = False # True for applying rolling avg

if rolling_avg_label:
    df[feature_columns] = df[feature_columns].rolling(window=360).mean()

In [ ]:
benign_data_starting_point = "2024-03-20 14:14:50.19"
benign_data_ending_point = "2024-03-23 16:26:19.00"


benign_filter_1 = (df['_time'].between(benign_data_starting_point, benign_data_ending_point))
benign_filter_2 = (~df['imeisv'].isin(['8642840401594200', '8642840401612300','8642840401624200','3557821101183501']))
benign_filter_3 = (df['label'] == 0)
benign_data_filter = (benign_filter_1 & benign_filter_2 & benign_filter_3)

In [ ]:
# benign data
benign_data_train = df[benign_data_filter].copy()
benign_data_train = benign_data_train.sort_values(['imeisv','_time'])
print(benign_data_train.shape[0])

260051


In [ ]:
benign_data_test_period_start = "2024-03-24 01:20:00.19"
benign_devices_for_testing = ['8609960468879057', '8628490433231157','8677660403123800']

benign_filter_4 = (df['_time'] >= benign_data_test_period_start)
benign_filter_5 = (df['imeisv'].isin(benign_devices_for_testing))
benign_data_filter_test = (benign_filter_3 & benign_filter_4 & benign_filter_5)

benign_data_test = df[benign_data_filter_test].copy()
benign_data_test = benign_data_test.sort_values(['imeisv','_time'])
print(benign_data_test.shape[0])

90102


In [ ]:
#malicious data
attck_1_start = "2024-03-23 21:26:00"
attck_1_end = "2024-03-23 22:23:00"
ues_to_exclude_in_1st_attck = [
    '8628490433231157','8609960480666910',
    '3557821101183501'] #'8677660403123800' '8642840401594200'

attck_2_start = "2024-03-23 22:56:00"
attck_2_end = "2024-03-23 23:56:00"
ues_to_exclude_in_2nd_attck = [
    '8609960480666910','8642840401612300'
]

mal_filter_1 = (
    df['_time'].between(attck_1_start, attck_1_end)
    & (~df['imeisv'].isin(ues_to_exclude_in_1st_attck))
)

mal_filter_2 = (
    df['_time'].between(attck_2_start, attck_2_end)
    & (~df['imeisv'].isin(ues_to_exclude_in_2nd_attck))
)

mal_filter_3 = (df['label'] == 1)

malicious_data = df[(mal_filter_1 | mal_filter_2) & mal_filter_3].copy()
malicious_data = malicious_data.sort_values(['imeisv','_time'])
print(malicious_data.shape[0])

10971


In [7]:
search_space_dict = {
    'window_size': [60, 90, 120], 
    'batch_size' : [32],
    'model_arch_config': [
        {
        'hidden_dim1' : 25, 
        'hidden_dim2' : 50, 
        },
        {
        'hidden_dim1' : 50, 
        'hidden_dim2' : 100, 
        },
        {
        'hidden_dim1' : 25, 
        'hidden_dim2' : 25, 
        },
        {
        'hidden_dim1' : 50, 
        'hidden_dim2' : 50, 
        }
    ],
    'dropout':[0.2, 0.3],
    'layer_norm_flag':[True, False],
    'loss_function' : [nn.L1Loss, nn.MSELoss], 
    'lr' : [1e-4, 1e-3],
    'num_epochs':[52]
    
}

In [8]:
for window_size, batch_size in itertools.product(search_space_dict['window_size'], search_space_dict['batch_size']):
    for config in search_space_dict['model_arch_config']:
        hidden_dim1 = config['hidden_dim1']
        hidden_dim2 = config['hidden_dim2']
        for dropout, layer_norm_flag, loss_function, lr, num_epochs in itertools.product(
            search_space_dict['dropout'], search_space_dict['layer_norm_flag'],
            search_space_dict['loss_function'], search_space_dict['lr'],
            search_space_dict['num_epochs']):
            
            experiment_id = str(uuid.uuid4())
            
            parameters = {
                'window_size': window_size,
                'step_size': int(window_size/3),
                'batch_size': batch_size,
                'hidden_dim1': hidden_dim1,
                'hidden_dim2': hidden_dim2,
                'dropout': dropout,
                'layer_norm_flag': layer_norm_flag,
                'loss_function': loss_function,
                'lr': lr,
                'num_epochs': num_epochs
            }
            
            print("##########################################################")
            print_parameters(parameters, experiment_id)
            
            train_data_loader, val_data_loader, mal_data_loader = create_ds_loader(
                benign_data_train, 
                malicious_data, 
                parameters['window_size'], 
                parameters['step_size'], 
                feature_columns, 
                parameters['batch_size']
            )
            
            rae_model = LSTMAutoencoder(
                input_dim = len(feature_columns), 
                hidden_dim1 = hidden_dim1, 
                hidden_dim2 = hidden_dim2, 
                output_dim = len(feature_columns), 
                dropout = dropout, 
                layer_norm_flag = layer_norm_flag
            )

            rae_model.to(device)

            early_stopping = EarlyStopping(patience=7, min_delta=0.)
            criterion = loss_function()
            
            history = rae_model.train_model(
                num_epochs = parameters['num_epochs'], 
                early_stopping = early_stopping, 
                train_data_loader = train_data_loader, 
                val_data_loader = val_data_loader, 
                mal_data_loader = mal_data_loader, 
                device = device, 
                criterion = criterion,  
                lr = lr
            )
            
            parameters['loss_function'] = parameters['loss_function'].__name__
            
            additional_metadata = {
                "rolling_avg": rolling_avg_label,
                "feature_columns": feature_columns,
                'dataset_used': dataset_used
            }
            
            
            save_experiment_results(history, parameters, additional_metadata, experiment_id, results_dir='../results')
            
            del rae_model
            torch.cuda.empty_cache()

##########################################################
Experiment edce45d0-6a35-42b3-be51-14b28246bd74 Parameters:

Window size: 60
Step size: 20
Batch size: 32
Hidden dim1: 25
Hidden dim2: 50
Dropout: 0.2
Layer norm flag: Enabled
Loss function: <class 'torch.nn.modules.loss.L1Loss'>
Lr: 0.0001
Num epochs: 52


Validating...: 100%|██████████| 83/83 [00:00<00:00, 879.38it/s]


Epoch 1, Train Loss: 0.6010, Val Loss: 0.6955


Validating...: 100%|██████████| 83/83 [00:00<00:00, 880.91it/s]


Epoch 2, Train Loss: 0.5799, Val Loss: 0.6916


...

Epoch 49, Train Loss: 0.7221, Val Loss: 0.7778


Validating...: 100%|██████████| 83/83 [00:00<00:00, 904.29it/s]


Epoch 50, Train Loss: 0.7192, Val Loss: 0.7744


Validating...:  96%|█████████▋| 80/83 [00:00<00:00, 889.10it/s]


KeyboardInterrupt: 